## Multi-label classification

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.conv_learner import *

In [ ]:
display(torch.cuda.is_available())
display(torch.backends.cudnn.enabled)

In [ ]:
PATH = f'/home/{os.environ["USER"]}/data/loc-newspaper/'
# images_root = f'{data_root}images/'


In [ ]:
ls {PATH}

In [ ]:
from fastai.plots import plots_from_files

In [ ]:
list_paths = [f'{PATH}images-modified/0016af005540341d896923265a80c3b6.jpg', f'{PATH}images-modified/002d4d8d6a00aea0eb5c0db25feef011.jpg']
titles=["unknown", "unknown"]
plots_from_files(list_paths, titles=titles, maintitle="Sample images")

In [ ]:
label_csv = f'{PATH}labels.csv'

In [ ]:
import pandas as pd
pd.read_csv(label_csv).head()

In [ ]:
n = len(list(open(label_csv))) - 1
val_idxs = get_cv_idxs(n)

> Try removing labels with NaN

In [ ]:
def get_data(sz, bs=64):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.05)
    return ImageClassifierData.from_csv(
        PATH,
        'images-modified',
        label_csv,
        bs=bs,
        tfms=tfms,
        suffix='.jpg',
        val_idxs=val_idxs
    )

In [ ]:
arch = resnet34

> Try 168, 225, 299

In [ ]:
sz = 168
bs = 16
data = get_data(sz, bs)

In [ ]:
# data = data.resize(int(sz * 1.3), 'tmp') # if much smaller than orig images, resize images for speed later

In [ ]:
learn = ConvLearner.pretrained(arch, data, precompute=True)
# learn.loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
lrf = learn.lr_find()
learn.sched.plot()

In [ ]:
lr = 0.2 # choose lr where loss is still clearly improving

In [ ]:
learn.precompute=False
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.precompute=False
lrs = [lr / 6, lr / 3, lr]
learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.sched.plot_loss()

In [ ]:
learn.save('168')

In [ ]:
# learn.precompute=False
# learn.load('168')

In [ ]:
sz=224

learn.precompute=False
learn.set_data(get_data(sz, bs))
learn.freeze()

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.sched.plot_loss()

In [ ]:
learn.save('224')

In [ ]:
learn.precompute=False
learn.load('224')

Bump up size to 299

In [ ]:
sz=299

learn.precompute=False
learn.set_data(get_data(sz, bs))
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
lrs = [lr / 6, lr / 3, lr]
learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.sched.plot_loss()

In [ ]:
learn.save('299')

In [ ]:
from sklearn.metrics import fbeta_score

def f2(preds, targs, start=0.17, end=0.24, step=0.01):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return max([fbeta_score(targs, (preds>th), 2, average='samples')
                    for th in np.arange(start,end,step)])

In [ ]:
probs = learn.predict()
f2(probs, data.val_y)

In [ ]:
multi_preds, y = learn.TTA()
preds = np.mean(multi_preds, 0)

f2(preds,y)

### End